# Experiment: How good was the selection of contacted customers compared to a computed sampling set?

## Setup

In [ ]:
import sys
PROJECT_PATH = "/home/christopher_orlowicz1_vodafone_c/gershgorin"
sys.path.append(PROJECT_PATH)
%load_ext autoreload
%autoreload 2

In [ ]:
%cd $PROJECT_PATH

In [ ]:
#!pip install -q -r requirements.txt
#!pip install faiss-cpu==1.7.1
#!pip install faiss-gpu==1.7.1

In [ ]:
import time

from google.cloud import bigquery
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = "retina"
import networkx as nx
import numpy as np
import pandas as pd
import pandas_gbq
import scipy
from sklearn.neighbors import kneighbors_graph

import src.db.big_query as bq
from src.db.preprocessing import Preprocessor
from src.db.zip_code_mapper import ZipCodeMapper
from src.gershgorin.bs_gda import bs_gda
from src.graph.graph import Graph
from src.graph import metrics
from src.graph import sample_evaluation
from src.graph.nearest_neighbors import NearestNeighbors
import src.utils.plotting as util_plt

## Build customer graph

### Load the data

Earliest possible date is **2021-12-09** (occurence of first `answer_value`s in NPS table).

In [ ]:
from_date = "2022-11-01"
to_date = "2022-11-30"

In [ ]:
# load a random sample of the CAR data
car_extra_df = bq.car_query_timeframe_sample(from_date, to_date, limit=1000)
# load all available feedback scores
car_gt_df = bq.join_car_nps(from_date, to_date)
# concatenate both DataFrames
union = pd.concat([car_extra_df, car_gt_df]).reset_index(drop=True)

In [ ]:
car_extra_df.shape, car_gt_df.shape, union.shape

### Preprocessing

#### Removing unused features

In [ ]:
prep = Preprocessor(from_date, to_date, data=None, verbose=False)
car_df, client_ids, adr_zips = prep.car_df, prep.client_ids, prep.adr_zips

#### Removing unknown zip codes

In [ ]:
# load mapper for zip_code -> (longitude, latitude)
zip_mapper = ZipCodeMapper()

In [ ]:
# load zip codes of customers
adr_zip_df = pd.DataFrame(adr_zips, dtype=int)
# remove unknown (unmappable) zip codes
known_zips = adr_zip_df.adr_zip.isin(zip_mapper.zip_code_map.index)
# apply mask to all three Dataframes
adr_zips = adr_zip_df.loc[known_zips].reset_index(drop=True)
car_df = car_df[known_zips].reset_index(drop=True)
client_ids = client_ids[known_zips].reset_index(drop=True)
car_df.shape

#### Mapping zip codes to (long, lat) pairs

In [ ]:
# map zip code to coords
coords = zip_mapper.map_zip_codes_to_coords(adr_zip)
# remove zip codes, keep lat and long
coords.drop(columns="adr_zip", inplace=True)

### Graph construction

In [ ]:
import gc
from numba import cuda

gc.collect()
cuda.select_device(0)
cuda.close()

In [ ]:
# store data as tensor on GPU
X = torch.tensor(np.ascontiguousarray(car_df.to_numpy()), device=torch.device('cuda', 0), dtype=torch.float32)
# compute k-nearest neighbor graph
knn = NearestNeighbors(device="gpu")
t = time.perf_counter()
_, k_neighbors = knn.knn(X, k=100)
print(f"This took {time.perf_counter()-t:.3f} s")

In [ ]:
A = knn.to_adj_matrix(k_neighbors)
n_nodes = A.shape[0]

In [ ]:
# directed graph: count_nonzero(A) >= n_edges >= count_nonzero(A)/2
# undirected graph: count_nonzero(A)/2 = n_edges
n_edges = A.getnnz()
metrics.density(n_nodes, n_edges, mode="directed")

## Draw graph and highlight actually sampled nodes ("ground truth")

In [ ]:
actual_sampling_set = car_gt_df.client_id
# map all client ids to node ids
actual_set = np.flatnonzero(client_ids.isin(actual_sampling_set))

In [ ]:
# use different node colors for each sampling set
colors, labels = sample_evaluation.prepare_colors_labels(n_nodes, [], actual_set)

In [ ]:
# use geographical coordinates of customers for graph layout
fixed_zip_pos = coords.to_dict("index")
# map dict of dicts to dict of tuples
fixed_zip_pos = {key: (values["long"], values["lat"]) for key, values in fixed_zip_pos.items()}

In [ ]:
def circular_layout_around_zip_codes(fixed_zip_pos: dict, radius: float = 0.2):
    pos = dict()
    for node, coord in fixed_zip_pos.items():
        x, y = coord
        rand_x, rand_y = np.random.rand(2)
        pos[node] = [x + radius*np.cos(rand_x*2*np.pi), y + radius*np.sin(rand_y*2*np.pi)]
    return pos

In [ ]:
pos = circular_layout_around_zip_codes(fixed_zip_pos, radius=0.1)

In [ ]:
x = [x for x, y in pos.values()]
y = [y for x, y in pos.values()]
c = np.zeros(car_df.shape[0])
c[colors[2]] = 1
plt.figure(figsize=(10,10))
plt.scatter(x, y, s=5, c=c, cmap=plt.cm.copper)
plt.axis("off");
plt.savefig("graph_true_samples_2000000.png", dpi=300)

In [ ]:
# construct a networkx graph from the adjacency matrix
t = time.perf_counter()
#nx_graph = nx.from_scipy_sparse_matrix(A)
print(f"This took {time.perf_counter()-t:.3f} s")

In [ ]:
sample_evaluation.plot_sample_classes(nx_graph, colors, labels, size=10, 
                                      pos=pos, subgraph=True, select=[2], hide_edges=True, 
                                      figsize=(8,8))
plt.savefig("graph_true_samples_1000000.png", dpi=300)

## Compute sampling set

In [ ]:
#sampling_budget = 1000
sampling_budget = len(car_gt_df)
graph = Graph(A)
start = time.perf_counter()
sampling_set, thres = bs_gda(graph, sampling_budget, p_hops=6, parallel=True)
print(f"This took {time.perf_counter()-start:.3f} s")
print("Budget:", sampling_budget)
print("Sampled nodes:", len(sampling_set))

## Evaluate the result

### Get predicted and actually sampled customer ids

In [ ]:
pred_sampling_set = client_ids[sampling_set]  # map node_id to client_id
actual_sampling_set = car_gt_df.client_id

In [ ]:
# map all client ids to node ids
actual_set = np.flatnonzero(client_ids.isin(actual_sampling_set))
intersect_set = list(set(sampling_set) & set(actual_set))

#### Statistics

In [ ]:
print("Predicted samples:", len(pred_sampling_set))
print("Actually sampled:", len(actual_sampling_set))
print("Overlap:", len(intersect_set), f"({len(intersect_set)/len(actual_sampling_set)*100:.3f} % of all surveyed customers)")

## Draw the network

Use a different node color per sample.

In [ ]:
# construct an networkx graph from the adjacency matrix
nx_graph = nx.from_scipy_sparse_array(graph.adj)
# precompute a graph layout for plotting
pos = nx.spring_layout(nx_graph, iterations=15)

In [ ]:
colors, labels = sample_evaluation.prepare_colors_labels(n_nodes, sampling_set, actual_set)

#### Whole graph

In [ ]:
sample_evaluation.plot_sample_classes(nx_graph, colors, labels, size=20, 
                                      pos=pos, subgraph=False, hide_edges=True, figsize=(8,8))
plt.savefig("graph_with_samples.png", dpi=300)

#### Specific sample in whole graph

In [ ]:
sample_evaluation.plot_sample_classes(nx_graph, colors, labels, size=20, 
                                      pos=pos, subgraph=True, select=[2], hide_edges=True, 
                                      figsize=(8,8))
plt.savefig("subgraph.png", dpi=300)

#### Subgraphs

In [ ]:
#util_plt.draw_subgraph(nx_graph, sampling_set)

In [ ]:
#util_plt.draw_subgraph(nx_graph, actual_set)

In [ ]:
#util_plt.draw_subgraph(nx_graph, intersect_set)

### Compute smallest eigenvalue of coefficient matrix

In [ ]:
def compute_exact(L, sampling_set, mu=0.01):
    n_nodes = L.shape[0]
    a = np.zeros(n_nodes, dtype=bool)
    a[list(sampling_set)] = 1
    B = np.diag(a) + mu * L
    B = sparse.csc_matrix(B)
    se, _ = sparse.linalg.eigsh(B, k=1, which='SM')
    return se

In [ ]:
# compare smallest eigenvalues
lap = graph.laplacian()
smallest_eig_actual = compute_exact(lap, actual_set)
print(f"smallest eigenvalue via eigen-decomposition: {smallest_eig_actual[0]}\n")

smallest_eig_pred = compute_exact(lap, sampling_set)
print(f"smallest eigenvalue via eigen-decomposition: {smallest_eig_pred[0]}\n")

print("Estimated threshold:", thres)

### Compare MSE

In [ ]:
# read recommendation scores
answers_df = bq.nps_query_timeframe(from_date, to_date)
# build ground truth signal by removing answers that cannot be assigned to a customer in CAR
s = answers_df.copy()
s = s[s.client_id.isin(client_ids)]

In [ ]:
def reconstruction_matrix(L, sampling_set: list, mu=0.01):
    n_nodes = L.shape[0]
    # sampling matrix
    a = np.zeros(n_nodes, dtype=bool)
    a[list(sampling_set)] = 1
    # coefficient matrix
    B = np.diag(a) + mu * L
    B = sparse.csc_matrix(B)
    I = np.eye(n_nodes)
    # reconstruction matrix
    reconstr_mat = I - mu * sparse.linalg.inv(B) @ L
    return reconstr_mat

def reconstruct_signal(reconstr_matrix, s):
    return reconstr_matrix @ s.reshape(-1, 1)

def mse(a, b):
    return np.mean(np.square(a-b))

In [ ]:
L = graph.laplacian()
rec_mat_pred = reconstruction_matrix(L, sampling_set)
s_reconst = reconstruct_signal(rec_mat_pred, s)
print("MSE sampling set:", mse(s, s_reconst))

rec_mat_actual = reconstruction_matrix(L, actual_set)
s_reconst = reconstruct_signal(rec_mat_actual, s)
print("MSE actual set:", mse(s, s_reconst))

### Investigate neighborhood of sampled nodes

In [ ]:
def neighborhood(graph, sampling_set, actual_set, max_hops=12):
    """
    How large do we have to choose the neighborhood of the actually sampled nodes 
    to cover all nodes of the predicted sampling set?
    """
    n_nodes = len(graph.nodes)
    # mark actually sampled nodes as uncovered
    uncovered = np.zeros(n_nodes, dtype=bool)
    uncovered[actual_set] = 1
    depths = range(1, max_hops+1)
    for depth in depths:
        for node in sampling_set:
            # do a limited BFS
            neighborhood = list(nx.bfs_tree(graph, node, depth_limit=depth).nodes())
            # mark visited nodes
            uncovered[neighborhood] = 0
        if not any(uncovered):
            # if all nodes in actual_set are covered, return the current depth
            return depth
    # if not all nodes were covered, return -1
    return -1

In [ ]:
neighborhood(nx_graph, sampling_set, actual_set, max_hops=5)